In [1]:
import sys
import os
codebase = '../'
sys.path.append(codebase)

In [2]:
import numpy as np
import tensorflow as tf

In [3]:
from model import VRNNCell, VRNN_model
from data import get_stock_data
from sampler import create_train_test_dataset
from transformation import fhn, linear, lorenz
from distribution import dirac_delta, mvn
from trainer import trainer